In [1]:
import json
import torch
import commentjson
import numpy as np
import os
import gc
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    DataCollatorForTokenClassification
)
from datasets import Dataset, Features, Value, ClassLabel, Sequence
from sklearn.metrics import accuracy_score
from seqeval.metrics import f1_score, classification_report
import logging
from sklearn.model_selection import train_test_split

# --- 0. 기본 설정 ---
MODEL_NAME = "klue/bert-base"
MAX_LEN = 64
EPOCHS = 5
BATCH_SIZE = 4

# 모델 저장 경로 설정
INTENT_MODEL_DIR = "./models/intent"
NER_MODEL_DIR = "./models/ner"
INTENT_LABEL_PATH = os.path.join(INTENT_MODEL_DIR, "intent_labels.jsonc")
NER_LABEL_PATH = os.path.join(NER_MODEL_DIR, "ner_labels.jsonc")

# --- 1. 데이터 로드 ---
# 1.1 Intent 분류를 위한 데이터
with open('intent_label_list.jsonc', 'r', encoding='utf-8') as f:
    intent_label_list = commentjson.load(f)

intent_label_to_id = {label: i for i, label in enumerate(intent_label_list)}
intent_id_to_label = {i: label for label, i in intent_label_to_id.items()}

with open('intent_data.jsonc', 'r', encoding='utf-8') as f:
    intent_data = commentjson.load(f)

# 1.2 NER을 위한 데이터
with open('ner_data.jsonc', 'r', encoding='utf-8') as f:
    loaded_ner_data = commentjson.load(f)

ner_data = []
for item in loaded_ner_data:
    entities_as_tuples = [tuple(entity_list) for entity_list in item.get("entities", [])]
    ner_data.append({"text": item.get("text", ""), "entities": entities_as_tuples})

# --- 2. Intent 분류 모델 훈련 ---
def train_intent_model():
    print("\n" + "="*50)
    print("Intent 분류 모델 훈련 시작")
    print("="*50)

    global intent_label_list, intent_label_to_id, intent_id_to_label

    num_intent_labels = len(intent_label_list)
    print(f"Intent 레이블 ({num_intent_labels}개) 사용: {intent_label_to_id}")

    intent_features = Features({
        'text': Value('string'),
        'label': ClassLabel(num_classes=num_intent_labels, names=intent_label_list)
    })

    intent_dataset = Dataset.from_list(intent_data, features=intent_features)

    # 학습/평가 데이터 분리
    train_test_datasets = intent_dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset = train_test_datasets["train"]
    eval_dataset = train_test_datasets["test"]

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    def preprocess_intent_data(examples):
        tokenized = tokenizer(
            examples['text'],
            truncation=True,
            max_length=MAX_LEN
        )
        return {
            "input_ids": tokenized["input_ids"],
            "attention_mask": tokenized["attention_mask"]
        }

    tokenized_train_dataset = train_dataset.map(
        preprocess_intent_data,
        batched=True,
        remove_columns=['text']
    )

    tokenized_eval_dataset = eval_dataset.map(
        preprocess_intent_data,
        batched=True,
        remove_columns=['text']
    )

    print(f"Intent 데이터 샘플: {tokenized_train_dataset[0]}")

    intent_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    intent_model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=num_intent_labels,
        id2label=intent_id_to_label,
        label2id=intent_label_to_id
    )

    def compute_intent_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return {"accuracy": accuracy_score(labels, predictions)}

    training_args = TrainingArguments(
        output_dir="./results/intent",
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        logging_dir='./logs/intent',
        logging_steps=10,
        eval_steps=100,
        eval_strategy="steps",
        save_steps=100,
        save_total_limit=2,
        report_to="none",
    )

    intent_trainer = Trainer(
        model=intent_model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
        data_collator=intent_data_collator,
        compute_metrics=compute_intent_metrics,
    )
    intent_trainer.train()
    eval_result = intent_trainer.evaluate()
    print(f"Intent 모델 평가 결과: {eval_result}")
    print("Intent 모델 훈련 완료")

    os.makedirs(INTENT_MODEL_DIR, exist_ok=True)
    intent_model.save_pretrained(INTENT_MODEL_DIR, safe_serialization=False)
    tokenizer.save_pretrained(INTENT_MODEL_DIR)

    with open(INTENT_LABEL_PATH, 'w', encoding='utf-8') as f:
        json.dump({
            "id2label": intent_id_to_label,
            "label2id": intent_label_to_id
        }, f, ensure_ascii=False, indent=2)

    print(f"Intent 모델 및 레이블 정보 저장 완료: {INTENT_MODEL_DIR}")

    return intent_model, tokenizer, intent_id_to_label

# --- 3. NER 모델 훈련 (개선된 버전) ---
def train_ner_model():
    print("\n" + "="*50)
    print("NER 모델 훈련 시작 (개선된 버전)")
    print("="*50)

    # 3.1 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    # 디버깅: 토크나이저 테스트
    test_text = "인문학"
    test_tokens = tokenizer.tokenize(test_text)
    print(f"토크나이저 테스트 '{test_text}' -> {test_tokens}")

    # 3.2 NER 레이블 정의 (BIO 형식)
    # 먼저 모든 엔터티 타입 추출
    entity_types = sorted(list(set(label for item in ner_data for _, _, label in item["entities"])))
    print(f"찾은 엔터티 타입: {entity_types}")

    ner_labels = ["O"]  # Outside tag
    for entity_type in entity_types:
        ner_labels.extend([f"B-{entity_type}", f"I-{entity_type}"])

    ner_label2id = {label: i for i, label in enumerate(ner_labels)}
    ner_id2label = {i: label for label, i in ner_label2id.items()}

    print(f"NER 레이블 ({len(ner_labels)}개): {ner_labels}")

    # 3.3 개선된 NER 데이터 전처리 - 문자 단위 접근
    preprocessed_ner_data = []

    print("\n--- 개선된 NER 데이터 전처리 시작 ---")

    for example_idx, example in enumerate(ner_data):
        text = example["text"]
        entities = example["entities"]

        if example_idx < 3:  # 처음 몇 개 예시만 상세 출력
            print(f"\n[데이터 {example_idx}] 텍스트: \"{text}\"")
            print(f"  정의된 엔티티: {entities}")

        # 1. 문자 단위 BIO 태깅 초기화
        char_labels = ["O"] * len(text)

        # 2. 엔티티에 따라 BIO 태그 할당
        for start_char, end_char, entity_type in entities:
            for i in range(start_char, end_char):
                if start_char < 0:
                    start_char = 0
                if end_char > len(text):
                    end_char = len(text)

                if start_char < end_char and start_char < len(text):
                    for i in range(start_char, end_char):
                        if i == start_char:
                            char_labels[i] = f"B-{entity_type}"
                        else:
                            char_labels[i] = f"I-{entity_type}"

        if example_idx < 3 or start_char >= len(text) or end_char > len(text) or start_char < 0:
            print(f"  문제가 있는 엔티티: ({start_char}, {end_char}, {entity_type})")
            print(f"  텍스트 길이: {len(text)}")

        if example_idx < 3:  # 상세 디버깅 출력
            print(f"  문자별 BIO 태그:")
            for i, (char, label) in enumerate(zip(text, char_labels)):
                print(f"    '{char}': {label}")

        # 3. 토큰화 및 토큰-문자 정렬
        tokenized = tokenizer(text, return_offsets_mapping=True, add_special_tokens=True)
        tokens = tokenizer.convert_ids_to_tokens(tokenized['input_ids'])
        offset_mapping = tokenized['offset_mapping']

        if example_idx < 3:  # 상세 디버깅 출력
            print(f"  토큰화 결과: {tokens}")
            print(f"  오프셋 매핑: {offset_mapping}")

        # 4. 토큰별 레이블 할당
        token_labels = []
        for i, (start, end) in enumerate(offset_mapping):
            # 특수 토큰 처리
            if start == end:
                token_label = -100  # ignore_index
            else:
                # 토큰 시작 위치의 문자 레이블 사용
                char_label = char_labels[start]
                token_label = ner_label2id[char_label]

                # 서브워드 토큰은 I- 태그로 변환 (WordPiece ##로 시작하는 경우)
                if i > 0 and tokens[i].startswith("##"):
                    prev_label = token_labels[-1]
                    if prev_label != -100 and ner_id2label[prev_label].startswith("B-"):
                        # B- -> I- 변환
                        entity_type = ner_id2label[prev_label][2:]  # "B-genre" -> "genre"
                        token_label = ner_label2id[f"I-{entity_type}"]

            token_labels.append(token_label)

        if example_idx < 3:  # 상세 디버깅 출력
            print(f"  최종 토큰 레이블: {[ner_id2label.get(l, 'IGN') for l in token_labels]}")

        # 5. 레이블 ID로 변환
        preprocessed_ner_data.append({
            "text": text,
            "input_ids": tokenized["input_ids"],
            "attention_mask": tokenized["attention_mask"],
            "labels": token_labels
        })

    # 3.4 데이터셋 생성
    ner_features = Features({
        'text': Value('string'),
        'input_ids': Sequence(Value('int32')),
        'attention_mask': Sequence(Value('int32')),
        'labels': Sequence(Value('int32'))
    })

    ner_dataset = Dataset.from_list(preprocessed_ner_data, features=ner_features)

    # 학습/평가 데이터셋 분리
    train_test_datasets = ner_dataset.train_test_split(test_size=0.2, seed=42)
    train_dataset = train_test_datasets["train"]
    eval_dataset = train_test_datasets["test"]

    print(f"NER 훈련 데이터 크기: {len(train_dataset)}")
    print(f"NER 평가 데이터 크기: {len(eval_dataset)}")
    print(f"NER 데이터 샘플: {train_dataset[0]}")

    # 3.5 데이터 콜레이터 설정
    ner_data_collator = DataCollatorForTokenClassification(
        tokenizer=tokenizer,
        padding=True,
        max_length=MAX_LEN,
        pad_to_multiple_of=8
    )

    # 3.6 모델 로드
    ner_model = AutoModelForTokenClassification.from_pretrained(
        MODEL_NAME,
        num_labels=len(ner_labels),
        id2label=ner_id2label,
        label2id=ner_label2id
    )

    # 3.7 평가 지표 계산 함수
    def compute_ner_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        # 실제 토큰의 예측값과 레이블만 추출
        true_predictions = []
        true_labels = []

        for prediction, label in zip(predictions, labels):
            true_pred = []
            true_label = []

            for p, l in zip(prediction, label):
                if l != -100:  # -100은 무시
                    true_pred.append(ner_id2label[p])
                    true_label.append(ner_id2label[l])

            true_predictions.append(true_pred)
            true_labels.append(true_label)

        # seqeval의 f1_score 계산
        try:
            f1 = f1_score(true_labels, true_predictions)
            report = classification_report(true_labels, true_predictions, digits=4)
            print("\nNER Classification Report:\n", report)

            # 세부 클래스별 결과 분석
            class_results = {}
            for label in ner_labels:
                if label != "O" and label.startswith("B-"):
                    entity_type = label[2:]  # "B-genre" -> "genre"
                    label_f1 = f1_score([[label]], [[label]], average='macro')
                    class_results[entity_type] = label_f1

            # 상세 예측 예시 출력
            print("\n예측 예시:")
            for i in range(min(5, len(true_labels))):
                print(f"실제: {true_labels[i]}")
                print(f"예측: {true_predictions[i]}")
                print("---")

            return {
                "f1": f1,
                **{f"f1_{key}": val for key, val in class_results.items()}
            }
        except Exception as e:
            print(f"Error calculating NER metrics: {e}")
            return {"f1": 0.0}

    # 3.8 훈련 설정
    training_args = TrainingArguments(
        output_dir="./results/ner",
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        logging_dir='./logs/ner',
        logging_steps=10,
        save_steps=100,
        eval_steps=100,
        eval_strategy="steps",
        load_best_model_at_end=True,
        save_total_limit=2,
        greater_is_better=True,
        metric_for_best_model="f1",
        weight_decay=0.01,
        report_to="none",
    )

    # 3.9 Trainer 정의
    ner_trainer = Trainer(
        model=ner_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=ner_data_collator,
        compute_metrics=compute_ner_metrics,
    )

    # 3.10 모델 훈련
    ner_trainer.train()
    eval_result = ner_trainer.evaluate()
    print(f"NER 모델 평가 결과: {eval_result}")
    print("NER 모델 훈련 완료")

    # 3.11 모델 및 레이블 정보 저장
    os.makedirs(NER_MODEL_DIR, exist_ok=True)
    gc.collect()
    ner_model.cpu()
    ner_model.save_pretrained(NER_MODEL_DIR, safe_serialization=False)
    tokenizer.save_pretrained(NER_MODEL_DIR)

    # 레이블 매핑 저장
    with open(NER_LABEL_PATH, 'w', encoding='utf-8') as f:
        json.dump({
            "id2label": ner_id2label,
            "label2id": ner_label2id
        }, f, ensure_ascii=False, indent=2)

    print(f"NER 모델 및 레이블 정보 저장 완료: {NER_MODEL_DIR}")

    return ner_model, tokenizer, ner_id2label


# --- 4. 저장된 모델 로드 ---
def load_intent_model():
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(INTENT_MODEL_DIR)

    # 레이블 정보 로드
    with open(INTENT_LABEL_PATH, 'r', encoding='utf-8') as f:
        label_info = json.load(f)

    id2label = {int(k): v for k, v in label_info["id2label"].items()}
    label2id = label_info["label2id"]

    # 모델 로드
    model = AutoModelForSequenceClassification.from_pretrained(
        INTENT_MODEL_DIR,
        num_labels=len(id2label),
        id2label=id2label,
        label2id=label2id
    )

    return model, tokenizer, id2label


def load_ner_model():
    # 토크나이저 로드 - 저장된 위치에서 직접 로드
    tokenizer = AutoTokenizer.from_pretrained(NER_MODEL_DIR, use_fast=True)
    print(f"NER용 토크나이저 로드 완료 (타입: {type(tokenizer)})")

    # 레이블 정보 로드
    with open(NER_LABEL_PATH, 'r', encoding='utf-8') as f:
        label_info = json.load(f)

    id2label = {int(k): v for k, v in label_info["id2label"].items()}
    label2id = label_info["label2id"]

    # 모델 로드
    model = AutoModelForTokenClassification.from_pretrained(
        NER_MODEL_DIR,
        num_labels=len(id2label),
        id2label=id2label,
        label2id=label2id
    )

    return model, tokenizer, id2label


# --- 5. 예측 함수 ---
def predict_intent(model, tokenizer, id2label, text):
    model.eval()

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=MAX_LEN
    )

    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']

    if torch.cuda.is_available():
        model = model.to("cuda")
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class_id = torch.argmax(logits, dim=-1).item()
        predicted_intent = id2label[predicted_class_id]

    return {
        "intent": predicted_intent,
        "confidence": probabilities.cpu().numpy().flatten().tolist()
    }


def predict_entities(model, tokenizer, id2label, text):
    if not model or not tokenizer or not id2label:
        return {"tokens": [], "tags": [], "entities": [], "token_probabilities": []}

    model.eval()

    # 토큰화 시 오프셋 매핑 반환
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True,
                      max_length=MAX_LEN, return_offsets_mapping=True)

    # 오프셋 매핑 별도 저장 및 제거 (모델에 전달하지 않음)
    offset_mapping = inputs.pop("offset_mapping").cpu().numpy()[0]

    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']

    # GPU 사용 가능 시 이동
    if torch.cuda.is_available():
        model = model.to("cuda")
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=2)
        predictions = torch.argmax(logits, dim=2)

    # 결과 추출
    input_ids = inputs["input_ids"][0].cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    predicted_tag_ids = predictions[0].cpu().numpy()
    all_probabilities = probabilities[0].cpu().numpy()

    # 디버깅을 위한 출력
    print("-" * 30)
    print("predict_entities 내부 디버깅:")
    print(f"입력 텍스트: {text}")
    print(f"토큰화 결과: {tokens}")
    print(f"예측된 태그 ID: {predicted_tag_ids}")

    # 특수 토큰 제외 및 결과 처리
    content_tokens = []
    content_tags = []
    token_probabilities_list = []
    content_offsets = []

    for i, (token, tag_id, input_id, (start, end)) in enumerate(zip(tokens, predicted_tag_ids, input_ids, offset_mapping)):
        if input_id == tokenizer.pad_token_id:
            break

        if token not in tokenizer.all_special_tokens:
            content_tokens.append(token)
            tag = id2label.get(tag_id, "O")
            content_tags.append(tag)
            content_offsets.append((start, end))
            token_probabilities_list.append(all_probabilities[i].tolist())

    # 디버깅 출력 (일부)
    print(f"특수 토큰 제외 토큰: {content_tokens}")
    print(f"해당 토큰의 태그: {content_tags}")
    print(f"해당 토큰의 오프셋: {content_offsets}")

    # 엔티티 추출
    entities_found = []
    current_entity_tokens = []
    current_entity_starts = []
    current_entity_ends = []
    current_entity_label = None

    for i, (token, tag, (start, end)) in enumerate(zip(content_tokens, content_tags, content_offsets)):
        if tag.startswith("B-"):
            # 이전 엔티티 처리
            if current_entity_tokens:
                entity_text = text[current_entity_starts[0]:current_entity_ends[-1]]
                entities_found.append({
                    "entity": entity_text,
                    "label": current_entity_label
                })

            # 새 엔티티 시작
            current_entity_tokens = [token]
            current_entity_starts = [start]
            current_entity_ends = [end]
            current_entity_label = tag[2:]  # "B-genre" -> "genre"

        elif tag.startswith("I-") and current_entity_label == tag[2:]:
            # 기존 엔티티 계속
            current_entity_tokens.append(token)
            current_entity_starts.append(start)
            current_entity_ends.append(end)

        else:
            # 엔티티 종료
            if current_entity_tokens:
                entity_text = text[current_entity_starts[0]:current_entity_ends[-1]]
                entities_found.append({
                    "entity": entity_text,
                    "label": current_entity_label
                })
            current_entity_tokens = []
            current_entity_starts = []
            current_entity_ends = []
            current_entity_label = None

    # 마지막 엔티티 처리
    if current_entity_tokens:
        entity_text = text[current_entity_starts[0]:current_entity_ends[-1]]
        entities_found.append({
            "entity": entity_text,
            "label": current_entity_label
        })

    # 문자 단위 시각화를 위한 태그 매핑
    char_tags = ["_"] * len(text)

    for entity in entities_found:
        entity_text = entity["entity"]
        entity_type = entity["label"]

        # 원본 텍스트에서 엔티티 위치 찾기
        start_pos = text.find(entity_text)
        if start_pos != -1:
            end_pos = start_pos + len(entity_text)

            # 위치에 태그 표시 (첫 글자는 B-, 나머지는 I-)
            char_tags[start_pos] = "B"
            for i in range(start_pos + 1, end_pos):
                char_tags[i] = "I"

    print(f"추출된 엔티티: {entities_found}")
    print(f"문자별 태그 매핑: {''.join(char_tags)}")
    print("-" * 30)

    return {
        "tokens": content_tokens,
        "tags": content_tags,
        "entities": entities_found,
        "token_probabilities": token_probabilities_list,
        "char_tags": char_tags  # 시각화를 위한 문자 단위 태그
    }


# --- 6. 통합 예측 함수 ---
def predict(text):
    global intent_model, intent_tokenizer, intent_id2label, ner_model, ner_tokenizer, ner_id2label

    if not intent_model or not ner_model:
        return {"text": text, "intent": "오류: 모델 로드 실패", "confidence": 0.0, "entities": [], "error": "모델 로드 실패"}
    try:
        intent_result = predict_intent(intent_model, intent_tokenizer, intent_id2label, text)
        ner_result = predict_entities(ner_model, ner_tokenizer, ner_id2label, text)

        # 문자별 태그 시각화 추가
        char_tags = ner_result.get("char_tags", ["_"] * len(text))

        return {
            "text": text,
            "intent": intent_result["intent"],
            "confidence": max(intent_result["confidence"]) if intent_result.get("confidence") else 0.0,
            "entities": ner_result["entities"],
            "ner_tokens": ner_result.get("tokens", []),
            "ner_token_probabilities": ner_result.get("token_probabilities", []),
            "char_tags": "".join(char_tags)  # 문자 단위 시각화
        }
    except Exception as e:
        import traceback
        print(f"예측 중 오류 발생: {e}")
        traceback.print_exc()
        return {"text": text, "intent": "예측 오류", "confidence": 0.0, "entities": [],
                "ner_tokens": [], "ner_token_probabilities": [], "error": str(e)}


# --- 7. 모델 훈련 함수 ---
def train_models():
    print("\n" + "="*50)
    print("도서 검색 NLU 모델 훈련 시작")
    print("="*50)

    # 모델 훈련 및 저장
    train_intent_model()
    train_ner_model()

    print("\n" + "="*50)
    print("도서 검색 NLU 모델 훈련 완료")
    print("="*50)

In [2]:
train_models()


도서 검색 NLU 모델 훈련 시작

Intent 분류 모델 훈련 시작
Intent 레이블 (40개) 사용: {'search_book_title': 0, 'search_book_author': 1, 'search_book_location': 2, 'check_book_availability': 3, 'get_bestseller': 4, 'get_new_releases': 5, 'request_recommendation_genre': 6, 'request_recommendation_mood': 7, 'request_recommendation_topic': 8, 'request_recommendation_similar': 9, 'request_recommendation_reader': 10, 'search_space_availability': 11, 'reserve_space': 12, 'get_space_info': 13, 'check_space_reservation': 14, 'cancel_space_reservation': 15, 'search_program': 16, 'apply_program': 17, 'get_program_info': 18, 'check_program_application': 19, 'cancel_program_application': 20, 'get_library_info': 21, 'inquire_service': 22, 'manage_membership': 23, 'check_loan_status': 24, 'extend_loan': 25, 'reserve_book': 26, 'check_reservation_status': 27, 'cancel_book_reservation': 28, 'check_overdue_status': 29, 'report_lost_item': 30, 'greeting': 31, 'gratitude': 32, 'closing': 33, 'affirmative': 34, 'negative': 35, 'ab

Map:   0%|          | 0/549 [00:00<?, ? examples/s]

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

Intent 데이터 샘플: {'label': 0, 'input_ids': [2, 10571, 2024, 1052, 3630, 636, 1644, 1513, 6076, 1560, 2918, 13964, 16, 3915, 1556, 1897, 2223, 2477, 1295, 1513, 2075, 2182, 35, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\spring\CaterpillarAi\venv\Lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(



NER Classification Report:
                    precision    recall  f1-score   support

           author     0.9265    0.9545    0.9403        66
         category     1.0000    1.0000    1.0000         1
             date     0.5000    0.5000    0.5000         2
         duration     0.0000    0.0000    0.0000         1
       event_type     1.0000    0.5000    0.6667         2
           format     0.9577    0.9855    0.9714        69
library_info_type     0.0000    0.0000    0.0000         1
         location     0.8000    1.0000    0.8889         4
        publisher     0.0000    0.0000    0.0000         1
  target_audience     1.0000    1.0000    1.0000         1
             time     0.0000    0.0000    0.0000         0
        timeOfDay     0.0000    0.0000    0.0000         1
            title     0.8607    0.9130    0.8861       115
            topic     0.0000    0.0000    0.0000         1

        micro avg     0.8971    0.9208    0.9088       265
        macro avg     0.5

c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\spring\CaterpillarAi\venv\Lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
c:\spring\CaterpillarAi\venv\Lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`

c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\spring\CaterpillarAi\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



NER Classification Report:
                    precision    recall  f1-score   support

           author     0.9552    0.9697    0.9624        66
         category     1.0000    1.0000    1.0000         1
             date     0.5000    0.5000    0.5000         2
         duration     0.0000    0.0000    0.0000         1
       event_type     1.0000    1.0000    1.0000         2
           format     0.9577    0.9855    0.9714        69
library_info_type     0.0000    0.0000    0.0000         1
         location     0.8000    1.0000    0.8889         4
        publisher     0.0000    0.0000    0.0000         1
  target_audience     1.0000    1.0000    1.0000         1
             time     0.0000    0.0000    0.0000         0
        timeOfDay     0.0000    0.0000    0.0000         1
            title     0.8689    0.9217    0.8945       115
            topic     0.0000    0.0000    0.0000         1

        micro avg     0.9081    0.9321    0.9199       265
        macro avg     0.5

In [4]:
intent_model, intent_tokenizer, intent_id2label = load_intent_model()
ner_model, ner_tokenizer, ner_id2label = load_ner_model()

NER용 토크나이저 로드 완료 (타입: <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>)


In [ ]:
from ipywidgets import widgets
from IPython.display import display, clear_output

text_widget = widgets.Text(description='질문:', placeholder='질문을 입력하세요')
button = widgets.Button(description='예측')
output = widgets.Output()

def on_button_click(b):
    global ner_id2label

    with output:
        clear_output(wait=True)
        user_input = text_widget.value
        if not user_input:
            print("질문을 입력해주세요.")
            return

        print(f"입력: \"{user_input}\"")
        print("예측 수행 중...")
        result = predict(user_input)

        print(f"\n{'='*60}")
        if "error" in result:
            print(f"🚨 예측 오류: {result['error']}")
        else:
            print(f"📄 입력 텍스트: \"{result['text']}\"")
            print(f"🎯 의도 분석 결과: {result['intent']} (확률: {result.get('confidence', 0.0):.4f})")

            print("\n🔍 개체명 인식 결과:")
            if result.get('entities'):
                for i, entity in enumerate(result['entities'], 1):
                    print(f"  {i}. \"{entity['entity']}\" → {entity['label']}")
            else:
                print("  인식된 개체명 없음")

            print("\n📊 토큰별 NER 확률:")
            if result.get('ner_tokens') and result.get('ner_token_probabilities'):
                if 'ner_id2label' in globals() and ner_id2label:
                    for token, probs in zip(result['ner_tokens'], result['ner_token_probabilities']):
                        print(f"  - 토큰: '{token}'")
                        sorted_probs = sorted(enumerate(probs), key=lambda item: item[1], reverse=True)
                        for tag_id, prob in sorted_probs:
                            print(f"      {ner_id2label.get(tag_id, f'ID_{tag_id}')}: {prob:.4f}")
                else:
                    print("  (오류: NER 레이블 매핑 정보(ner_id2label)를 찾을 수 없습니다.)")
            else:
                print("  토큰별 확률 정보 없음.")

            print("\n📝 문자별 NER 태그 시각화:")
            print(f"  원문: {result['text']}")

            text = result['text']
            tag_markers = ['_'] * len(text)
            processed_indices = set()
            for entity in sorted(result.get('entities', []), key=lambda x: len(x['entity']), reverse=True):
                entity_text = entity['entity']
                entity_type = entity['label']
                start_pos = -1
                search_start = 0
                while True:
                    temp_pos = text.find(entity_text, search_start)
                    if temp_pos == -1: break
                    is_overlapping = False
                    for i in range(temp_pos, temp_pos + len(entity_text)):
                        if i in processed_indices:
                            is_overlapping = True
                            break
                    if not is_overlapping:
                        start_pos = temp_pos
                        break
                    search_start = temp_pos + 1

                if start_pos != -1:
                    end_pos = start_pos + len(entity_text)
                    for i in range(start_pos, end_pos):
                        if i not in processed_indices:
                            tag_markers[i] = '^'
                            processed_indices.add(i)
                    tag_label = f"[{entity_type}]"
                    for i, char in enumerate(tag_label):
                        pos = start_pos + i
                        if pos < len(tag_markers) and tag_markers[pos] == '^':
                            tag_markers[pos] = char

            print(f"  태그: {''.join(tag_markers)}")


button.on_click(on_button_click)
display(text_widget, button, output)

Text(value='', description='질문:', placeholder='질문을 입력하세요')

Button(description='예측', style=ButtonStyle())

Output()